## Imports

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

## About the Data

**Your goal is to predict the operating condition of a waterpoint for each record in the dataset.**
 
- amount_tsh - Total static head (amount water available to waterpoint)
- date_recorded - The date the row was entered
- funder - Who funded the well
- gps_height - Altitude of the well
- installer - Organization that installed the well
- longitude - GPS coordinate
- latitude - GPS coordinate
- wpt_name - Name of the waterpoint if there is one
- num_private -
- basin - Geographic water basin
- subvillage - Geographic location
- region - Geographic location
- region_code - Geographic location (coded)
- district_code - Geographic location (coded)
- lga - Geographic location
- ward - Geographic location
- population - Population around the well
- public_meeting - True/False
- recorded_by - Group entering this row of data
- scheme_management - Who operates the waterpoint
- scheme_name - Who operates the waterpoint
- permit - If the waterpoint is permitted
- construction_year - Year the waterpoint was constructed
- extraction_type - The kind of extraction the waterpoint uses
- extraction_type_group - The kind of extraction the waterpoint uses
- extraction_type_class - The kind of extraction the waterpoint uses
- management - How the waterpoint is managed
- management_group - How the waterpoint is managed
- payment - What the water costs
- payment_type - What the water costs
- water_quality - The quality of the water
- quality_group - The quality of the water
- quantity - The quantity of water
- quantity_group - The quantity of water
- source - The source of the water
- source_type - The source of the water
- source_class - The source of the water
- waterpoint_type - The kind of waterpoint
- waterpoint_type_group - The kind of waterpoint

In [ ]:
train_values = pd.read_csv("data/train_set_values.csv")
train_labels = pd.read_csv("data/train_set_labels.csv")

train_df = pd.merge(train_values, train_labels, on="id")
print(train_df.shape)
train_df.head()

In [ ]:
train_df["population"].value_counts()

In [ ]:
loc_cols = ["district_code", "ward"]


# Invalid coordinates
bad_lat = -2.000000e-08
bad_lon = 0.000000

mask = (train_df["latitude"] == bad_lat) & (train_df["longitude"] == bad_lon)
group_means = (
    train_df.loc[~mask]  # exclude bad values
    .groupby(loc_cols)[["latitude", "longitude"]]
    .mean()
    .reset_index()
)

In [ ]:
group_means

In [ ]:
train_df["permit"].value_counts()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df["public_meeting"].value_counts()

In [ ]:
train_df["funder"].fillna(train_df["funder"].mode().values[0], inplace=True)

In [ ]:
train_df[train_df["longitude"] == 0][["latitude", "longitude"]]

In [ ]:
train_df["installer"].value_counts()

In [ ]:
train_df[["funder", "installer"]]

## Target

In [ ]:
target_vcs = train_df["status_group"].value_counts()

sns.set_theme('paper')
sns.barplot(x=target_vcs.keys(), y=target_vcs, hue=target_vcs.keys())
plt.title("Distribution of Target")
plt.xlabel("Status Group")
plt.ylabel("Count")
plt.show()

**Inference:** The target value is skewed with 'functional needs repair' having the lest amount of samples. Techniques to handle imbalanced data will be required -> Oversampling, undersampling, weightage.

In [ ]:
train_df.columns

In [ ]:
top_n = 5
vcs = train_df["source"].value_counts()
top_vcs = vcs.head(top_n)
replace_w_other = lambda x: x if x in top_vcs.index else 'Other'
train_df["source"] = train_df["source"].apply(replace_w_other)
train_df.source.value_counts()

In [ ]:
dummies = pd.get_dummies(train_df.source, prefix="source_", drop_first=True)
dummies.head()


In [ ]:
train_df = pd.concat([train_df, dummies], axis=1)
train_df.head()

In [ ]:
numerical_cols = ["amount_tsh", "gps_height", "longitude", "latitude", "population", "num_private"]
categorical_cols = ["funder", "installer", "wpt_name", "basin", "subvillage", "region", "region_code", "district_code", "lga", "ward", "public_meeting", "scheme_management", "scheme_name", "permit", "extraction_type", "extraction_type_group", "extraction_type_class", "management", "management_group", "payment", "payment_type", "water_quality", "quality_group", "quantity", "quantity_group", "source", "source_type", "source_class", "waterpoint_type", "waterpoint_type_group", "recorded_by"]
date_cols = ['construction_year', 'date_recorded']
target = "status_group"
remaining_cols = set(train_df.columns) - set(numerical_cols) - set(categorical_cols) - set(date_cols) - set([target])
print(remaining_cols)

## Distribution

In [ ]:

for col in numerical_cols:
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))  

    # Original Histogram
    sns.histplot(train_df[col], kde=True, ax=axes[0, 0], color='skyblue')
    axes[0, 0].set_title(f"Original Distribution of {col}")
    axes[0, 0].set_xlabel(col.capitalize())

    # Log Histogram (only positive values)
    sns.histplot(np.log(train_df[train_df[col] > 0][col]), kde=True, ax=axes[0, 1], color='salmon')
    axes[0, 1].set_title(f"Log Distribution of {col}")
    axes[0, 1].set_xlabel(f"log({col})")

    # Original Violinplot
    sns.violinplot(x=train_df[col], ax=axes[1, 0], color='skyblue')
    axes[1, 0].set_title(f"Original Violinplot of {col}")
    axes[1, 0].set_xlabel(col.capitalize())

    # Log Violinplot (only positive values)
    sns.violinplot(x=np.log(train_df[train_df[col] > 0][col]), ax=axes[1, 1], color='salmon')
    axes[1, 1].set_title(f"Log Violinplot of {col}")
    axes[1, 1].set_xlabel(f"log({col})")

    plt.tight_layout()
    plt.show()

- amount_tsh: lots of values near 0: log transform
- gps_height: lots of values near 0: not right so fix with median?
- lat and long: fix 0 values with region with external api
- population: fix 0 (why would you build well near 0 population): also log transform
- num_private: no info in website so idk

In [ ]:

top_n = 20  # for many-category columns

for col in categorical_cols:
    n_unique = train_df[col].nunique()
    
    if n_unique <= 20:
        # Simple vertical bar plot
        plt.figure(figsize=(8, 5))
        vcs = train_df[col].value_counts().sort_values(ascending=False)
        sns.barplot(x=vcs.index, y=vcs.values, palette="viridis")
        plt.xticks(rotation=45, ha="center")
        plt.title(f"Distribution of {col}", fontsize=14)
        plt.xlabel(col, fontsize=12)
        plt.ylabel("Count", fontsize=12)
        plt.tight_layout()
        plt.show()
        
    else:
        # Many categories → horizontal bar plot, top N + "Other"
        vcs = train_df[col].value_counts()
        top_vcs = vcs.head(top_n)
        others_sum = vcs.iloc[top_n:].sum()
        top_vcs['Other'] = others_sum
        
        plt.figure(figsize=(10, 6))
        sns.barplot(x=top_vcs.values, y=top_vcs.index, palette="viridis")
        plt.xlabel("Count")
        plt.ylabel(col)
        plt.title(f"Top {top_n} {col} Categories + Other")
        plt.xscale("log")  # log scale for better visibility of small counts
        plt.tight_layout()
        plt.show()

- Lots of categories (collapse into other): >= 20?
- Locations (only use lat and long): subvillage, region, region_code, district_code, lga, ward, 

In [ ]:
train_df["latitude"].value_counts()

In [ ]:
train_df["longitude"].value_counts()

In [ ]:
train_df[["latitude", "longitude"]].value_counts()

In [ ]:
-2.000000e-08

In [ ]:
loc_cols = ["subvillage", "region", "region_code", "district_code", "lga", "ward"]


# Define bad values
bad_lat = -2.000000e-08
bad_lon = 0.000000

# Mask for rows with invalid coordinates
mask = (train_df["latitude"] == bad_lat) & (train_df["longitude"] == bad_lon)

# Compute group-wise means (ignoring bad values)
group_means = (
    train_df.loc[~mask]  # exclude bad values
    .groupby(loc_cols)[["latitude", "longitude"]]
    .mean()
    .reset_index()
)

# Merge means back into original df
train_df = train_df.merge(group_means, on=loc_cols, how="left", suffixes=("", "_mean"))

# Replace only invalid coords with group means
train_df.loc[mask, "latitude"] = train_df.loc[mask, "latitude_mean"]
train_df.loc[mask, "longitude"] = train_df.loc[mask, "longitude_mean"]

# Drop helper columns
train_df.drop(columns=["latitude_mean", "longitude_mean"], inplace=True)

In [ ]:
train_df[["latitude", "longitude"]].value_counts()

## Date

In [ ]:
train_df[date_cols].head()

- One obvious feature is how long ago was the pump constructed

In [ ]:
train_df["construction_year"].value_counts()[:10]

- But most pumps do not have recorded construction date
- We can use median imputer per region and management

In [ ]:
# Replace 0 with NaN
train_df['construction_year'] = train_df['construction_year'].replace(0, np.nan)

# Group medians (some groups may still become NaN if all values were 0)
group_medians = (
    train_df.groupby(['region', 'management'])['construction_year']
    .transform('median')
)

# Fill missing with group median first
train_df['construction_year'] = train_df['construction_year'].fillna(group_medians)

# Fill any remaining NaNs with global median
global_median = train_df['construction_year'].median()
train_df['construction_year'] = train_df['construction_year'].fillna(global_median)

In [ ]:
train_df[date_cols].isna().sum()

In [ ]:
train_df[date_cols].head()

In [ ]:
train_df["construction_year"].value_counts()

In [ ]:
train_df["construction_year"] = train_df["construction_year"].astype(int)

In [ ]:
train_df["date_recorded"] = pd.to_datetime(train_df["date_recorded"], format="%Y-%m-%d")

In [ ]:
train_df["age"] = train_df["construction_year"] - train_df["date_recorded"].dt.year

## Correlation

In [ ]:
train_df_encoded = train_df.copy(deep=True)

for col in categorical_cols  + [target]:
    train_df_encoded[col] = LabelEncoder().fit_transform(train_df_encoded[col])

In [ ]:
corr = train_df_encoded[numerical_cols + categorical_cols + ["age"] + [target]].corr()

plt.figure(figsize=(12, 10))

# Optional: mask upper triangle for cleaner look
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, annot=False, cmap="coolwarm", mask=mask, linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

- lat and long correlated: probably because of 0 values
- installer and funder: probably because the person who funds also installs
- extraction_type and extraction_type_group and extraction_type_class: basically the same thing with more granularity: only use one
- source/source_type correlated with all extraction: Maybe since soil is similar in the source -> we use the same extraction methodd?
- payment_type and payment
- management_group and management
- quality and quality_group
- source and source_type
- waterpoint_type and water_point_type_group
- very low correlation of all features with targer col